# Visualize LIDAR Readings of Julian

In [1]:
from datetime import datetime
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from generate_atmosphere import LidarProfile
%matplotlib inline
import mpld3
mpld3.enable_notebook()

In [2]:
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)
rc('font', family='serif')

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
          #'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':10,
         'ytick.labelsize':10 }
pylab.rcParams.update(params)

#import plotly.plotly as py
#import plotly.tools as tls
#import plotly.graph_objs as go
#
#from plotly.offline import iplot, init_notebook_mode
#init_notebook_mode()

In [3]:
eps = np.finfo(np.float).eps
print (eps)
find_nearest = lambda array,value: (np.abs(array - value)).argmin()

2.220446049250313e-16


In [4]:
def smooth(x,window_len=11,window='hanning'):
    """smooth the data using a window with requested size.

    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.

    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal

    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)

    see also: 

    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter

    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    if x.ndim != 1:
        raise ValueError, "smooth only accepts 1 dimension arrays."

    if x.size < window_len:
        raise ValueError, "Input vector needs to be bigger than window size."


    if window_len<3:
        return x


    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"


    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y


SyntaxError: invalid syntax (<ipython-input-4-3f7daa61ea25>, line 34)

## Lidar model: $P(r) = P_0 \frac{c\tau}{2} \frac{A}{r^2} {\beta(r)}exp^{-2\int_0^r\sigma(r')dr'}$ 

$P_0$ - the transmitted power at time $t_0$ -  $[Watts]$  

$r=\frac{c(t-t_0)}{2}$ - hight - $[Km]$

$c$ - speed of light    - $[\frac{Km}{sec}]$ 

A - the effective system recieving area - $[Km^2]$

$\beta(r)$ - backscatter coefficient - $[\frac{1}{Km\cdot sr}]$

$\sigma(r)$  - extiction coefficient - $[\frac{1}{Km\cdot sr}]$

$\tau$ - Lidar pulse time - $[sec]$ 

TODO:  - check units and values of $P,P_0,A,\tau$


In [ ]:
def generate_P (P0,c,A,heights,alpha,beta):
    dr = heights[1:]-heights[0:-1] # dr for integration
    last = dr[-1]
    dr = np.append(dr,last)
    tau = np.mean(dr)*2/c  #[sec]  Is this resonable ? How to set the real value 
    lidar_const = 0.5*P0*c*tau*A
    numerator =  lidar_const*beta*np.exp(-2*np.cumsum(alpha*dr))
    denominator = np.power(heights,2)+eps # epsilon is to avoid NaN
    z_pos =  np.where(numerator == 0)[0]
    exp_alpha = np.exp(-2*np.cumsum(alpha*dr))
    P = numerator/denominator
    return P

# KLETT's inversion form: #
## $\sigma (r) = \frac{exp^{\frac{S-S_m}{k}}}{\sigma_m^{-1}+\frac{2}{k}\int_{r}^{r_m}{exp^{\frac{S-S_m}{k}}}dr}$
## $\beta(r)= B_0\alpha(r)^{\gamma}$ 

$S(r)=ln[r^2P(r)]$ - Logaritmic range adjusted power 

$r_m$ - reference range - How to set this values ? 

$S_m = S(r_m)$

$\sigma_m = \sigma(r_m)$  - How $\sigma_m$ is calculated from measurments? is it a kind of assumption ? 

$B_0$ - backscatter-to-extiction ratio - How this ratio is calculated?

$\gamma$ - a fitting coefficient $0.67<\gamma<1$ - How to set this coefficient?


In [ ]:
def calc_S (heights,P):
    '''Calculate the Logaritmic range adjusted power S(r) = ln(r^2*P(r))'''
    lanpow = lambda r,p: np.log(np.power(r,2)*p)#+eps) #if (h!=0 and p!=0) else 0
    S = np.array([lanpow(r,p) for (r,p) in zip(heights,P)])
    return S

def calc_extiction_klett(S, heights, alpha, ind_m):
    '''Calculate the inversion of the extinction coefficient'''
    k=1 
    S_m = S[ind_m]
    sigma_m = alpha[ind_m]
    
    dr =  heights[1]-heights[0]
    exp_S = np.exp((S-S_m)/k)
    denominator = (1/(sigma_m) + (2/k)*np.flip(np.cumsum(np.flip(exp_S,0)*dr),0))
    sigma = exp_S/(denominator)#+eps)
    return sigma

## Work on the 22 Apr

In [ ]:
# Load data
#heights = np.linspace(min_height, 10,num=100)
lidar_paths = sorted(glob.glob("../Lidar/2017_04_22/*41smooth.txt"))[:4]
profiles_alpha = []
profiles_beta = []

for path in lidar_paths:
    lidar_profile = LidarProfile(path)
    min_height = lidar_profile._lidar_height
    heights = np.linspace(min_height, 10,num=100)
    cur_profiles_0= lidar_profile.interpolate(heights)[0]
    cur_profiles_1= lidar_profile.interpolate(heights)[1]

    profiles_alpha.append(cur_profiles_0)
    profiles_beta.append(cur_profiles_1)

lidar_df = pd.read_csv(
    path,
    delimiter="\t",
    index_col=0,
    skiprows=None
)
lidar_df[lidar_df<0] = 0
times = []
for path in lidar_paths:
    s = path.split("/")[-1].split('-')[1:3]
    times.append("{}:{}".format(s[0].split('_')[1], s[1]))

In [ ]:
lidar_df

In [ ]:
'''Extinction coefficient'''
data_alpha = {k: v for (k, v) in zip(times, profiles_alpha)}
df_alpha = pd.DataFrame(index=heights, data=data_alpha, columns=times)
df_alpha.plot(figsize=(8,5))

'''Backscatter coefficient'''
data_beta = {k: v for (k, v) in zip(times, profiles_beta)}
df_beta = pd.DataFrame(index=heights, data=data_beta, columns=times)
df_beta.plot(figsize=(8, 5))

In [ ]:
# Visualize data 

beta = profiles_beta[0]
alpha = profiles_alpha[0] 

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,5))
ax = axes.ravel()

ax[0].plot(heights,alpha)
ax[0].set_ylabel('sigma')#, fontsize=18,fontweight='bold')# [km^-1sr^1]$")
ax[0].set_xlabel('Heights [km]')#,fontsize=18,fontweight='bold')
#ax[0].set_title('Extinction coefficient from Lidar measurments',fontsize=20,fontweight='bold')
ax[1].plot(heights,beta)
ax[1].set_ylabel('beta')#,fontsize=18,fontweight='bold')# [km^-1sr^1]$")
ax[1].set_xlabel('Heights [km]')#,fontsize=18,fontweight='bold')
#ax[1].set_title('Backscatter coefficient from Lidar measurments',fontsize=20,fontweight='bold')

plt.show()

#fig.savefig('orig_coeffs.svg',bbox_inches='tight')

In [ ]:
A = 1   # TODO : ask for this value 
P0 = 100  # TODO : ask for this value 
c = 299792 #[Km/sec]
alpha = profiles_alpha[0] # extiction coefficient
beta = profiles_beta[0] # backscater - adding eps for numerical issues
#print np.min(alpha),np.min(beta)
beta[beta<eps]= eps
alpha[alpha<eps] = eps
print np.min(alpha),np.min(beta),len(alpha),len(beta)

In [ ]:
# Calculate and show the power and the Logaritmic range adjusted power
P = generate_P(P0,c,A,heights,alpha,beta)
P[P<eps] = eps
S = calc_S (heights,P)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,5))
ax = axes.ravel()

ax[0].plot(heights,P)
ax[0].set_ylabel('Lidar Power')#, fontsize=18,fontweight='bold')# [km^-1sr^1]$")
ax[0].set_xlabel('Heights [km]')#,fontsize=18,fontweight='bold')
P_max = 1.1*np.max(P[1:-1])
P_min = -0.2e-3#-1.1*np.min(P[1:-1])
ax[0].set_ylim((P_min,P_max)) 
ax[0].set_xlim((0,np.max(heights))) 
#ax[0].set_title('Extinction coefficient from Lidar measurments',fontsize=20,fontweight='bold')

ax[1].plot(heights,S)
ax[1].set_ylabel('Logaritmic range adjusted power')#,fontsize=18,fontweight='bold')# [km^-1sr^1]$")
ax[1].set_xlabel('Heights [km]')#,fontsize=18,fontweight='bold')
#ax[1].set_title('Backscatter coefficient from Lidar measurments',fontsize=20,fontweight='bold')
plt.show()

#fig.savefig('generated_power.svg',bbox_inches='tight')


In [ ]:

print 'P: min-',np.min(P), ', max-',np.max(P), ', pos min-',np.where(P == P.min()), ', pos max-',np.where(P == P.max())
print 'S: min-',np.min(S), ', max-',np.max(S), ', pos min-',np.where(S <-30), ', pos max-',np.where(S == S.max())
#print heights[1:]-heights[0:-1]

In [ ]:
#grad_vals = np.gradient(S)
#high_grads = np.argwhere(np.abs(grad_vals)>1)
#print grad_vals
#print np.abs(grad_vals)>1
#print high_grads
#plt.plot(heights,grad_vals)
#plt.ylabel('gradient of S')
#plt.xlabel('Heights[Km]')
#plt.title('gradients of Logaritmic range adjusted power')
#print S[3],S[24]

ind_s = 3
ind_m = 24
r_s = heights[ind_s]
r_m = heights[ind_m]
S_m = S[ind_m]
find_nearest( heights, r_m )
print (heights[49])
#print beta/alpha


In [ ]:
#Calculate and show the inversion of the extinction and backscatter coefficients
ind_m = 49   # How to set the height rm and the values Sm ?? 46-49 doesn't work
print (heights[ind_m])
sigma = calc_extiction_klett(S, heights, alpha, ind_m)
B_0  = 0.018 #0.18 # np.mean(np.ma.masked_invalid(beta/(alpha+eps))) # backscatter-to-extiction ratio - for the retrieval 
k=1
beta_ = B_0*np.power(sigma,k)


In [ ]:
ind = np.where(alpha>10*eps)
print ind

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,5))
ax = axes.ravel()
ax[0].plot(heights,sigma)
ax[0].set_ylabel('sigma retrival')# $[\frac{1}{Km\cdot sr}]$')
ax[0].set_xlabel('Heights [Km]')
#ax[0].set_ylim((0,0.14))
#ax[0].set_xlim((0,10))
#ax[0].set_title('Extixtion coeefficient retrival')

ax[1].plot(heights,beta_)
ax[1].set_ylabel('beta retrival')
ax[1].set_xlabel('Heights [Km]')
ax[0].set_xlim((0,heights.max())) 
ax[0].set_ylim((0,sigma.max()*1.1)) 

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,5))
ax = axes.ravel()
ax[0].plot(heights,sigma)
ax[0].set_ylabel('sigma retrival')# $[\frac{1}{Km\cdot sr}]$')
ax[0].set_xlabel('Heights [Km]')
#ax[0].set_ylim((0,0.14))
#ax[0].set_xlim((0,10))
#ax[0].set_title('Extixtion coeefficient retrival')

ax[1].plot(heights,beta_)
ax[1].set_ylabel('beta retrival')
ax[1].set_xlabel('Heights [Km]')

In [ ]:
# TODO  - understands how to initialize : rm,sigma_m
# Why there is a factor of X0.1 from the retrievals
# Why the recovered sigma is reveresed to the original one ? 


## GD
$P(r) = P_0 \frac{c\tau}{2} \frac{A}{r^2} {\beta(r)}exp^{-2\int_0^r\sigma(r')dr'}$ 

$\beta(r)= B_0\alpha(r)^{\gamma}$

Assuming: $\gamma=1$, $B_0=0.17$, $P_{lidar}= P_0 \frac{c\tau}{2}A$. 

Then: 

y(r) = log($\frac{P(r)r^2}{P_{lidar}B_0}) = log({\sigma(r)}) {-2\int_0^r\sigma(r')dr'}$ 

The unknowns are $X = \{\sigma(r)\}_{r_1}^{r_n}$

The measurments are $Y = \{y(r)\}_{r_1}^{r_n}$

$g(X)=Y-log(IX)+2DX$, $D = dr\cdot L$, $L$- Lower triangular matrix of ones.

$\nabla_xg(X) =  -IX^{-1} + 2D$

The loss function: 

$f(x)=\frac{1}{2}||{g(X)}||^2$

$\nabla_xf(X) =  \nabla_xg(X)g(X)$






In [ ]:
n = len (heights)
dr = np.mean(heights[1:]-heights[0:-1]) # dr for integration
tau = dr*2/c  #[sec]  Is this resonable ? How to set the real value 
lidar_const = 0.5*P0*c*tau*A

# model function
D = dr*np.tril(np.ones((n,n)),0)
I = np.eye(n)
g = lambda X,Y:Y - np.log(X) + 2*np.dot(D,X)
grad_g = lambda X : -np.diag(1/X)+2*D

# cost function
f = lambda X,Y: 0.5*(g(X,Y)**2).mean()
grad_f = lambda X,Y: np.dot(grad_g(X),g(X,Y))/len(X)

# setting problem data - Y, and initial solution-X_0
X_0 = np.copy(sigma) #alpha 

Y = (np.power(heights,2)*P)/(lidar_const*B_0)
plt.figure(figsize=(8,5))
plt.plot(heights,Y)
plt.ylabel('Y')
plt.xlabel('Heights [Km]')
plt.title('The measurments Y used in GD')
print np.min(Y),np.min(sigma), np.min(P),np.min(alpha)

In [ ]:
win =7
d= (win-1)/2
X_0_smooth = np.copy(X_0)
X_0_smooth[50:-1] = smooth(X_0_smooth[50:-1], window_len=win)[d:-d]
plt.plot(heights,X_0)
plt.plot(heights,X_0_smooth)
plt.ylabel('X')#, fontsize=18,fontweight='bold')# [km^-1sr^1]$")
plt.xlabel('Heights [km]')#,fontsize=18,fontweight='bold')
#print 'P: min-',np.min(P), ', max-',np.max(P), ', pos min-',np.where(P == P.min()), ', pos max-',np.where(P == P.max())
#print 'P: min-',np.min(P_smooth), ', max-',np.max(P_smooth), ', pos min-',np.where(P_smooth == P_smooth.min()), ', pos max-',np.where(P_smooth == P_smooth.max())

In [ ]:
#win = 7
#d= (win-1)/2
#X_0_orig = np.copy(X_0)
#X_0 = smooth(X_0, window_len=win)
#X_0=X_0[d:-d]
#
#plt.figure(figsize=(8,5))
#plt.plot(heights,sigma)
#plt.plot(heights,X_0)
#plt.ylabel('Y')
#plt.xlabel('Heights [Km]')
#plt.title('The measurments Y used in GD')
#
#'''check this using a smooth signal'''

#beta_orig = np.copy(beta)
#alpha_orig = np.copy(alpha)
#
#win = 7
#d= (win-1)/2
#alpha_smooth = smooth(alpha, window_len=win)
#beta_smooth = smooth(beta, window_len=win)
#beta_smooth = smooth(beta, window_len=7)
#beta = beta_smooth[d:-d]
#alpha=alpha_smooth[d:-d]
#print len(beta_smooth), len(beta),len(beta_orig)

In [ ]:
'''Gradient Descent'''
rate = 1e-6
minb = 1e-7#eps - less then 1e-7 the GD is not converting
maxb = 1
epochs = 10000
loss_epochs = np.zeros(epochs)
grad_epochs =  np.zeros(epochs)
X = X_0_smooth #np.random.normal(0,1,n)+eps.
X[X<minb]=minb
verbose =  False
temporal_epochs = np.array([0,1000,5000,10000])
X_temp = np.zeros((len(temporal_epochs),n))
X_temp[0][:]=X
err = np.zeros((len(temporal_epochs),n))
err[0][:]= np.abs(g(X,Y))

for epoch in range(epochs):
    loss = f(X,Y)
    loss_epochs[epoch]=loss
    
    grad = grad_f(X,Y)
    grad_epochs[epoch]=np.linalg.norm(grad,2) # save the gradient norm
    # print np.linalg.norm(grad,2)  # The norm of the gradient is huge . check this issue.
    X = X - rate*grad
    X[X<minb]=minb # projected gradient descents
    X[0]=X[1]
    X[-1]=X[-2]
    X[X>maxb]=np.mean(X)#maxb  
    if np.any(temporal_epochs==epoch+1):       
        pos = np.argwhere(temporal_epochs == epoch+1)[0][0]
        X_temp[pos][:]=X
        err[pos][:]=np.abs(g(X,Y))
    if verbose and ((epoch+1) % 500 == 0):
        print ('epoch {}, loss {}'.format(epoch,loss))
#print ('epoch {}, loss {}, nex X {}'.format(epoch,loss,X))
#plt.figure()



In [ ]:
Epochs = np.arange(1, epochs + 1 , 1)
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,5))
ax = axes.ravel()

#plt.figure(figsize=(5,5))
ax[0].plot(Epochs, loss_epochs)
ax[0].set_ylabel('Loss')
ax[0].set_xlabel('Epochs no.')
ax[0].set_title('Projected GD loss')

#ax[1].plot(Epochs[2:-1], grad_epochs[2:-1])
#ax[1].set_ylabel('Gradient norm')
#ax[1].set_xlabel('Epochs no.')
#ax[1].set_title('Loss gradient norm')
#plt.figure(figsize=(8,5))
for i,epoch in enumerate(temporal_epochs):    
    ax[1].plot(heights,X_temp[i][:], label = 'epoch {}'.format(epoch) )
#plt.plot(heights,X, 'c--', label = 'X - GD - 100 epochs')

ax[1].plot(heights,alpha, 'b--', label = 'Original sigma')
ax[1].legend()
ax[1].set_ylabel('Sigma retrival')
ax[1].set_xlabel('Heights [Km]')
ax[1].set_title('GD sigma retrival over time')
ax[1].set_ylim((0,0.14))
ax[1].set_xlim((0,10))

#fig.savefig('GD_1e-7_smooth_X0.svg',bbox_inches='tight')

In [ ]:
plt.figure(figsize=(8,5))
for i,epoch in enumerate(temporal_epochs):    
    plt.plot(heights,X_temp[i][:], label = 'epoch {}'.format(epoch) )
#plt.plot(heights,X, 'c--', label = 'X - GD - 100 epochs')

plt.plot(heights,alpha, 'b--', label = 'X orig')
plt.legend()
plt.ylabel('GD sigma retrival')
plt.xlabel('Heights [Km]')
plt.title('GD sigma retrival over time')
plt.ylim((0,0.14))
plt.xlim((0,10))

plt.show()
#fig.savefig('GD_loss_sigma_nosmooth.svg',bbox_inches='tight')

In [ ]:
Epochs = np.arange(1, epochs + 1 , 1)
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,5))
ax = axes.ravel()


for i,epoch in enumerate(temporal_epochs):    
    ax[0].plot(heights,err[i][:], label = 'epoch {}'.format(epoch))
    diff_x = np.abs((X_temp[i][:]-alpha))/(alpha+eps)
    diff_x[diff_x>1]=0
    ax[1].plot(heights,diff_x, label = 'epoch {}'.format(epoch))
    
    
#ax[0].legend(loc=2)
ax[0].set_ylabel('Error')
ax[0].set_xlabel('Heights [Km]')
ax[0].set_title('Error g(X,Y) through time')
#
ax[1].set_ylabel('Error')
ax[1].set_xlabel('Heights [Km]')
ax[1].set_title('Error |X-Xorig|/Xorig through time')
#ax[1].set_ylim((0,100))



#ax[1].set_title('Error abs(X-X_0rig) through time')
#

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,5))
ax = axes.ravel()

for i,epoch in enumerate(temporal_epochs):  
    sigma_i = X_temp[i][:]
    beta_i = sigma_i*B_0
    ax[0].plot(heights,sigma_i, label = 'epoch {}'.format(epoch) )
    ax[1].plot(heights,beta_i, label = 'epoch {}'.format(epoch) )

ax[0].set_ylabel('GD sigma retrival')# $[\frac{1}{Km\cdot sr}]$')
ax[0].set_xlabel('Heights [Km]')
ax[1].set_ylabel('GD beta retrival')
ax[1].set_xlabel('Heights [Km]')
#ax[0].set_ylim((0,0.14))
#ax[0].set_xlim((0,10))
#ax[0].set_title('Extixtion coeefficient retrival')

## Alternating GD 
$P(r) = P_0 \frac{c\tau}{2} \frac{A}{r^2} {\beta(r)}exp^{-2\int_0^r\sigma(r')dr'}$ 

$\beta(r)= B_0\sigma(r)^{\gamma}$

Asuming: $\gamma=1$, $B_0=0.17$, $P_{lidar}= P_0 \frac{c\tau}{2}A$. 

Then: 

$P(r) = P_{lidar}B_0 \frac{1}{r^2} {\sigma(r)}exp^{-2\int_0^r\sigma(r')dr'}$ 

### First gradient step:

$y_1(r) = \frac{P(r)}{P_{lidar}B_0} = \frac{\sigma(r)}{r^2} \exp^{-2\int_0^r\sigma(r')dr'} $ 

#### Numeric form:

Define lower triangular matrix: $D = dr\cdot L$ ; $L$ is a lower triangular matrix of 1's 

Define vector: $e = \exp^{-2DX}\quad s.t.\quad   e_i = \exp^{-2 dr\cdot\sum_{j=1}^i\sigma_j^{(-)}}$ - $e$ is a constant vector based on prev solution of $\sigma^{(-)}$

$y_{1,i} =  \frac{\sigma_i}{r_i^2} \exp^{-2 dr\cdot\sum_{j=1}^i\sigma_j} = \frac{e_i}{r_i^2}\cdot \sigma_i$

The unknowns are $X = \{\sigma(r)\}_{r_1}^{r_n}= \{\sigma_i\}_{1}^{n}$

The measurments are $Y_1 = \{y_1(r)\}_{r_1}^{r_n} = \{y_{1,i}\}_{1}^{n}$

Define vector: $r = D1_{n\times1}= \{r_i\}_{1}^{n} = \{\sum_{j=1}^{n} D_{i,j}\}_{1}^{n} $

Define diagonal matrix: $R=diag(r)diag(r)= diag(\{r_i^2\}_{1}^{n})$

Define diagonal matrix: $E=diag(e)$ 

Define diagonal matrix: $Q = R^{-1}E$

Then: $Y_1 = R^{-1}EX= QX$

$g_1(X)=Y_1-QX$ 








$\nabla g_1(X) =  -Q $

###  Second gradient update:

$y_2(r) = \log\big(\frac{r^2P(r)}{P_{lidar}B_0\sigma(r)}\big) = -2\int_0^r\sigma(r')dr'$ 

#### Numeric form:

$y_{2,i} = \log\big(\frac{r_i^2P_i}{P_{lidar}B_0\sigma_i}\big)=-2 dr\cdot\sum_{j=1}^i\sigma_j$

Define diagonal matrix: $S=diag\big(\sigma^{(-)}\big)$ 

$Y_2 = \log(RS^{-1}Y_1 )= -2DX$

$g_2(X)=Y_2+2DX$, 

$\nabla g_2(X) = 2D$

### The loss function: 

$f_i(X)=\frac{1}{2}||{g_i(X)}||^2\quad i=1,2$

$\nabla f_i(X) =  \nabla g_i(X)g_i(X)\quad i=1,2$

### The update step in iteration $\kappa$

$X^{(\kappa,1)} \leftarrow \mathbf{P}\big(X^{(\kappa)}-\mu_1\nabla f_1(X^{(\kappa)})\big)$

$X^{(\kappa,2)} \leftarrow \mathbf{P}\big(X^{(\kappa,1)}-\mu_2\nabla f_2(X^{(\kappa,1)})\big)$

$X^{(\kappa+1)} \leftarrow X^{(\kappa,2)}$

    

In [ ]:
def f1(P,X):
    E = np.diagflat(np.exp(-2*np.dot(D,X)),k=0)
    Q = lidar_const*np.dot(E,np.linalg.inv(R))
    Y = (P)/(lidar_const*B_0)
    
    # model function
    g = np.dot(Q,X)-Y
    grad_g = Q
    
    # cost function - 1 
    f = 0.5*(g**2).mean()
    grad_f = np.dot(grad_g,g)/len(X)
    return [f,grad_f]

def f2(P,X):
    S = np.diagflat(X)
    S_inv = np.linalg.inv(S)
    P = (P)/(lidar_const*B_0)
    Y = np.log(np.dot(np.dot(R,S_inv),P))
    
    # model function
    g = 2*np.dot(D,X)-Y
    grad_g = 2*D
    
    # cost function - 1 
    f = 0.5*(g**2).mean()
    grad_f = np.dot(grad_g,g)/len(X)
    return [f,grad_f]

In [ ]:
#X = 0.1*np.ones((n,1))
D = dr*np.tril(np.ones((n,n)),0)
D[:,0]=heights[0]
rr =  np.power(heights,2) # rr = r^2 #one_v = np.ones((n,1)) #  np.array(np.dot(D,one_v)) 
R = np.diagflat(rr)

In [ ]:
'''Gradient Descent'''
rate = 1e-6
minb = 1e-7#eps - less then 1e-7 the GD is not converting
maxb = 1
epochs = 10000
loss1_epochs = np.zeros(epochs)
loss2_epochs =  np.zeros(epochs)

X = np.copy(sigma)
X[X<minb] = minb

verbose =  False
temporal_epochs = np.array([0,1000,5000,10000])
X_temp = np.zeros((len(temporal_epochs),n))
X_temp[0][:]=X
#err = np.zeros((len(temporal_epochs),n))
#err[0][:]= np.abs(g(X,Y))

for epoch in range(epochs):
    [loss1,g1] = f1(X,P)
    loss1_epochs[epoch]=loss1
    X = X - rate*g1
    X[X<minb]=minb
    
    
    [loss2,g2] = f2(X,P)
    loss2_epochs[epoch]=loss2
    X = X - rate*g2
    X[X<minb]=minb # projected gradient descents
     
    if np.any(temporal_epochs==epoch+1):       
        pos = np.argwhere(temporal_epochs == epoch+1)[0][0]
        X_temp[pos][:]=X
        #err[pos][:]=np.abs(g(X,Y))
    if verbose and ((epoch+1) % 500 == 0):
        print ('epoch {}, loss {}'.format(epoch,loss))

In [ ]:
Epochs = np.arange(1, epochs + 1 , 1)
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12,5))
ax = axes.ravel()

#plt.figure(figsize=(5,5))
ax[0].plot(Epochs, loss1_epochs)
ax[0].set_ylabel('Loss 1')
ax[0].set_xlabel('Epochs no.')
ax[0].set_title('Projected GD loss')

ax[1].plot(Epochs, loss2_epochs)
ax[1].set_ylabel('Loss 1')
ax[1].set_xlabel('Epochs no.')
ax[1].set_title('Projected GD loss')


for i,epoch in enumerate(temporal_epochs):    
    ax[2].plot(heights,X_temp[i][:], label = 'epoch {}'.format(epoch) )
#plt.plot(heights,X, 'c--', label = 'X - GD - 100 epochs')

ax[2].plot(heights,alpha, 'b--', label = 'Original sigma')
ax[2].legend()
ax[2].set_ylabel('Sigma retrival')
ax[2].set_xlabel('Heights [Km]')
ax[2].set_title('GD sigma retrival over time')
#ax[1].set_ylim((0,0.14))
#ax[1].set_xlim((0,10))


In [ ]:
#plt.figure(figsize=(5,5))
plt.plot(Epochs, loss1_epochs)
plt.ylabel('Loss 1')
plt.xlabel('Epochs no.')
plt.title('Projected GD loss')



In [ ]:
loss1_epochs

In [ ]:

D = dr*np.tril(np.ones((n,n)),0)
rr =  np.power(D.sum(1),2) # rr = r^2 #one_v = np.ones((n,1)) #  np.array(np.dot(D,one_v)) 
R = np.diagflat(rr)

E = lambda X,D: np.diagflat(np.exp(-2*np.dot(D,X)),k=0)
Q = lambda E,R: lidar_const*np.dot(E,np.linalg.inv(R))
Y1 = lambda P :(P)/(lidar_const*B_0)

S = np.diagflat(X)
Y2 = lambda Y,R,S : np.log(np.dot(np.dot(R,np.linalg.inv(S)),Y))

X_0 = np.zeros((n,1))
E_0= E(X_0,D)   # k=0 initial step
Q_0 = Q(E_0,R) # k=0 initial step


# model function - 1
g1 = lambda X,Y,Q:Y - np.dot(Q,X)
grad_g1 = lambda X,Q : -Q

# cost function - 1 
f1 = lambda X,Y: 0.5*(g1(X,Y,Q)**2).mean()
grad_f1 = lambda X,Y: np.dot(grad_g1(X,Q),g1(X,Y,Q))/len(X)

# model function - 2
g2 = lambda X,Y: Y - 2*np.dot(D,X)
grad_g2 = lambda X : 2*D

# cost function
f = lambda X,Y: 0.5*(g2(X,Y)**2).mean()
grad_f = lambda X,Y: np.dot(grad_g2(X),g2(X,Y))/len(X)




# LS: min 0.5*||Y-AX||^2
# GD: min 

In [ ]:
win = 7
d= (win-1)/2
alpha_smooth = smooth(alpha, window_len=win)
beta_smooth = smooth(beta, window_len=win)
print len(alpha_smooth),  len(alpha)
plt.figure(figsize=(8,5))
plt.plot(heights,alpha,'r')
plt.plot(heights,alpha_smooth[d:-d],'m')
print np.min(alpha_smooth)
plt.figure(figsize=(8,5))
plt.plot(heights,beta,'r')
plt.plot(heights,beta_smooth[d:-d],'m')
print np.min(beta_smooth)

In [ ]:
#import matplotlib as mpl
#mpl.rcParams['text.usetex'] = True
#mpl.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command

In [ ]:
def objective(X,Y):
    g = Y - np.log(X) + 2*np.dot(D,X)
    f = 0.5*(g**2).mean()
    return f

bnds=(b,)*n

In [ ]:
import numpy as np
from scipy.optimize import minimize

# show initial objective
print('Initial Objective: ' + str(objective(X,Y)))

solution = minimize(objective,X,Y,method='BFGS',bounds=bnds)
X = solution.x

# show final objective
print('Final Objective: ' + str(objective(X,Y)))
